## Expert Evaluation
Conduct the expert evaluation by importing the json files from `data/3_submissions_with_categorized_feedback_jsons` into a new expert evaluation in the playground. Download the results of the completed expert evaluation as well as the evaluation configuration.
Save the downloaded files in the `data/4_expert_evaluation` directory.

### Read Expert Evaluation
Once you have saved the expert evaluation results and the config file in the `data/4_expert_evaluation` directory, run the following cell to read the expert evaluation results into a DataFrame.

In [ ]:
from service.json_service import read_expert_evaluation

data_dir = "data/4_expert_evaluation"

expert_evaluations = read_expert_evaluation(data_dir)
expert_evaluations["evaluation_type"] = "expert"

### Save the Expert Evaluation
Save the expert evaluation results to a CSV file for further analysis and processing in the following steps.

In [ ]:
expert_evaluations.to_csv("data/4_expert_evaluation/expert_evaluation.csv", index=False)

### (Optional) Splitting the Evaluation Config
If you want to do multiple, potentially incomplete evaluations, you can split the evaluation config into multiple configs. All configs will contain the same exercises, submissions, feedback suggestions, and metrics. However, the configs will have different exercise and submission orders.
Specifically, the configs will have a rotating order of exercises and either an ascending or descending order of submissions.

This allows different experts to evaluate the same exercises and submissions, but in a different order. This can help reduce bias in the evaluation process. Additionally, it enables the experts to hand in a incomplete evaluation early, therefore reducing the invested time per expert.

In [ ]:
import json
import os
import copy

evaluation_config_path = "data/4_expert_evaluation/evaluation_config.json"
evaluation_config_splitted_path = "data/4_expert_evaluation/evaluation_configs_splitted"

os.makedirs(evaluation_config_splitted_path, exist_ok=True)

evaluation_config = {}

with open(evaluation_config_path, "r") as file:
    evaluation_config = json.load(file)

evaluation_config["started"] = False
evaluation_config["expertIds"] = []
evaluation_config["mappings"] = {}

exercises = evaluation_config["exercises"]


def generate_config(starting_index: int, ascending: bool) -> None:
    new_config = copy.deepcopy(evaluation_config)
    new_config["name"] = (
        f"{evaluation_config['name']}_{'ascending' if ascending else 'descending'}_{starting_index}"
    )
    new_config["exercises"] = exercises[starting_index:] + exercises[:starting_index]

    for exercise in new_config["exercises"]:
        exercise["submissions"].sort(key=lambda x: x["id"], reverse=not ascending)

    new_config_path = f"{evaluation_config_splitted_path}/{new_config['name']}.json"
    with open(new_config_path, "w") as new_file:
        json.dump(new_config, new_file, indent=4)


for ascending in [True, False]:
    for starting_index in range(len(exercises)):
        generate_config(starting_index, ascending)